In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
import lightgbm as lgb

import xgboost as xgb

In [2]:
train = pd.read_csv('Train.csv.zip', compression='zip')
test = pd.read_csv('Test.csv.zip', compression='zip')

In [3]:
# Save the 'Id' column
train_ID = train['id']
test_ID = test['id']

# Now drop the 'Id' column since it's unnecessary for the prediction process.
train.drop("id", axis = 1, inplace = True)
test.drop("id", axis = 1, inplace = True)

In [4]:
# настройка параметров
train[['build_tech']] = train[['build_tech']].replace(np.nan, 0.5)
train[['metro_dist']] = train[['metro_dist']].replace(np.nan, 15.0)
train[['g_lift']] = train[['g_lift']].replace(np.nan, 0.5)

In [5]:
train['area_per_room'] = train['area'] / train['rooms']
test['area_per_room'] = test['area'] / test['rooms']

In [6]:
test[['build_tech']] = test[['build_tech']].replace(np.nan, 0.5)
test[['metro_dist']] = test[['metro_dist']].replace(np.nan, 15.0)
test[['g_lift']] = test[['g_lift']].replace(np.nan, 0.5)

In [7]:
train['street_id'] = train['street_id'].astype(str)
test['street_id'] = test['street_id'].astype(str)

In [8]:
X = train[[
#     'date', 
    'street_id', 
    'build_tech', 
    'floor', 
    'area', 'area_per_room',
    'rooms',
    'balcon', 
    'metro_dist', 
    'g_lift', 
    'n_photos', 
#     'kw1', 'kw2', 'kw3', 'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12', 'kw13'
]]
y = train[['price']]

In [9]:
test = test[[
    'street_id', 
    'build_tech', 
    'floor', 
    'area', 'area_per_room',
    'rooms',
    'balcon', 
    'metro_dist', 
    'g_lift', 
    'n_photos', 
]]

In [10]:
# X = pd.get_dummies(X)
# print(X.shape)

In [11]:
# Cross-validation with k-folds
n_folds = 3

def mae_cv(model, X, y, name):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    mae= -cross_val_score(model, X.values, y, scoring="neg_mean_absolute_error", cv = kf)
    print(name + " score: {:.4f}".format(mae.mean()))

In [12]:
knn1 = KNeighborsRegressor(n_neighbors=3) # 1554037.1373234342
# mae_cv(knn1, X, y, 'knn-3')

knn2 = KNeighborsRegressor(n_neighbors=10) # 1580934.9331793853
# mae_cv(knn2, X, y, 'knn-10')

rg1 = Ridge(alpha=0.01, random_state=0) # 1692388.0292463482
# mae_cv(rg1, X, y, 'rg1-0.01')

rg2 = Ridge(alpha=1.1, random_state=0) # 1692378.7748252843
# mae_cv(rg2, X, y, 'rg1-1.1')

rg3 = Ridge(alpha=100.1, random_state=0) # 1692401.9077769322
# mae_cv(rg3, X, y, 'rg1-100.1')

lasso1 = Lasso(alpha=0.01, random_state=0) # 1692388.1266646797
# mae_cv(lasso1, X, y, 'lasso-1.01')

lasso2 = Lasso(alpha=1.1, random_state=0) # 1692387.7184467418
# mae_cv(lasso2, X, y, 'lasso-1.1')

lasso3 = Lasso(alpha=100.1, random_state=0) # 1692371.585207761
# mae_cv(lasso3, X, y, 'lasso-100.1')

# knn-3 score: 1554022.4553
# knn-10 score: 1580582.9061
# rg1-0.01 score: 1693203.7011
# rg1-1.1 score: 1693203.9926
# rg1-100.1 score: 1693235.5966
# lasso-1.01 score: 1693203.6984
# lasso-1.1 score: 1693203.6945
# lasso-100.1 score: 1693203.3706

In [13]:
rf1 = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0) # 1360153.8352
# mae_cv(rf1 ,X, y, 'rf-100')

rf2 = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0) # 1354697.7572
# mae_cv(rf2 ,X, y, 'rf-1000')

In [33]:
gbm1 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, nthread=-1, objective='regression', random_state=0) 
# 1659428.4293
# mae_cv(gbm1 ,X, y, 'gbm1-2')

In [34]:
gbm2 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, nthread=-1, objective='regression') 
# # 
# mae_cv(gbm2 ,X, y, 'gbm1-5')

In [35]:
gb1 = GradientBoostingRegressor(n_estimators=100, random_state=0) # 1557468.3937
# mae_cv(gb1 , X, y, 'gbm1-100')

In [36]:
xgb1 = xgb.XGBRegressor(max_depth=13, n_estimators=5000, random_state =7) # 875145.9986
# mae_cv(xgb1 , X, y, 'xgb1-5000')

In [37]:
gb2 = GradientBoostingRegressor(n_estimators=17500, random_state=0) # 680773.1029
# mae_cv(gb2 , X, y, 'gbm2-17500')

In [14]:
def stack_pred(estimator, X1, y1, k):    
    def put_on_places(arr, elems, places):
        for i in range(0, len(elems)):
            arr[places[i]] = elems[i]
        
    sX = [3] * len(X)    
    kf = KFold(n_splits=k, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X1):
        X_train, X_test = X1.values[train_index], X1.values[test_index]
        y_train, y_test = y1.values[train_index], y1.values[test_index]
        estimator.fit(X_train, y_train)
        pred = estimator.predict(X_test) 
        
        put_on_places(sX, pred, test_index)  
    return np.array(sX)

In [15]:
columns = ['knn1', 'knn2', 'rg1', 'rg2', 'rg3', 'lasso1', 'lasso2', 'lasso3', 'rf1', 'rf2', 'gbm1', 'gbm2', 'gb1', 'gb2']

In [16]:
df = pd.DataFrame(index=range(0, y.shape[0]), columns=columns)

In [17]:
df['knn1'] = stack_pred(knn1, X, y, 3)

In [18]:
df['knn2'] = stack_pred(knn2, X, y, 3)

In [19]:
df['rg1'] = stack_pred(rg1, X, y, 3)

In [20]:
df['rg2'] = stack_pred(rg2, X, y, 3)

In [21]:
df['rg3'] = stack_pred(rg3, X, y, 3)

In [22]:
df['lasso1'] = stack_pred(lasso1, X, y, 3)

In [23]:
df['lasso2'] = stack_pred(lasso2, X, y, 3)

In [24]:
df['lasso3'] = stack_pred(lasso3, X, y, 3)

In [25]:
df['rf1'] = stack_pred(rf1, X, y, 3)

/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()
/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()
/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


In [26]:
df['rf2'] = stack_pred(rf2, X, y, 3)

/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()
/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()
/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


In [38]:
df['gbm1'] = stack_pred(gbm1, X, y, 3)

/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [39]:
df['gbm2'] = stack_pred(gbm2, X, y, 3)

/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [40]:
df['gb1'] = stack_pred(gb1, X, y, 3)

/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
df['gb2'] =  0.2 * stack_pred(xgb1, X, y, 3) + 0.8 * stack_pred(gb2, X, y, 3)

/home/axreldable/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
# df

In [42]:
ens_model = GradientBoostingRegressor(n_estimators=200, random_state=0)

In [43]:
df_test = pd.DataFrame(index=range(0, y.shape[0]), columns=columns)

In [ ]:
knn1.fit(X, y)
knn2.fit(X, y)
rg1.fit(X, y)
rg2.fit(X, y)
rg3.fit(X, y)
lasso1.fit(X, y)
lasso2.fit(X, y)
lasso3.fit(X, y)
rf1.fit(X, y)
rf2.fit(X, y)

/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
X_for_gbm = X[[
#     'street_id', 
    'build_tech', 
    'floor', 
    'area', 'area_per_room',
    'rooms',
    'balcon', 
    'metro_dist', 
    'g_lift', 
    'n_photos', 
]]

In [ ]:
gbm1.fit(X_for_gbm, y)
gbm2.fit(X_for_gbm, y)

In [ ]:
xgb1.fit(X_for_gbm, y)

In [ ]:
gb1.fit(X, y)
gb2.fit(X, y)

In [ ]:
# columns = ['knn1', 'knn2', 'rg1', 'rg2', 'rg3', 'lasso1', 'lasso2', 'lasso3', 'rf1', 'rf2', 'gbm1', 'gbm2', 'gb1', 'gb2']
df_test['knn1'] = knn1.predict(test)
df_test['knn2'] = knn2.predict(test)
df_test['rg1'] = rg1.predict(test)
df_test['rg2'] = rg2.predict(test)
df_test['rg3'] = rg3.predict(test)
df_test['lasso1'] = lasso1.predict(test)
df_test['lasso2'] = lasso2.predict(test)
df_test['lasso3'] = lasso3.predict(test)
df_test['rf1'] = rf1.predict(test)
df_test['rf2'] = rf2.predict(test)

In [ ]:
test_for_gbm = test[[
#     'street_id', 
    'build_tech', 
    'floor', 
    'area', 'area_per_room',
    'rooms',
    'balcon', 
    'metro_dist', 
    'g_lift', 
    'n_photos', 
]]

In [ ]:
df_test['gbm1'] = gbm1.predict(test_for_gbm)
df_test['gbm2'] = gbm2.predict(test_for_gbm)

In [ ]:
df_test['gb1'] = gb1.predict(test)
# df_test['gb2'] = gb2.predict(test)
df_test['gb2'] = 0.2 * xgb1.predict(test_for_gbm) + 0.8 * gb2.predict(test)

In [ ]:
# df_test

In [ ]:
ens_model.fit(df, y)

In [ ]:
pred_rez = ens_model.predict(df_test)

In [ ]:
rez_test = test

In [ ]:
rez_test['price'] = pred_rez
rez_test['id'] = test_ID.values

rez_test[['id', 'price']].to_csv('sub.csv', index=False)

In [ ]:
# 